In [1]:
!cat ./info.txt

https://github.com/darksigma/Fundamentals-of-Deep-Learning-Book


In [2]:
!ls .

03_nns_in_tf.py.ipynb            dropout_regularization.py
04_beyond_gradient_descent.ipynb info.txt
05_CNN.py.ipynb                  input_data.py
__pycache__                      inverted_dropout.py
conv_mnist_logs                  logistic_logs
data                             neuron.py


In [3]:
import input_data
import tensorflow as tf
import time

In [4]:
mnist = input_data.read_data_sets('data/', one_hot=True)

Extracting data/train-images-idx3-ubyte.gz
Extracting data/train-labels-idx1-ubyte.gz
Extracting data/t10k-images-idx3-ubyte.gz
Extracting data/t10k-labels-idx1-ubyte.gz


In [5]:
# Architecture
n_hidden_1 = 256
n_hidden_2 = 256

# Hyperparameters
ETA = 0.0001
EPOCHS = 1000
BATCH = 100
DISPLAY_STEP = 1

In [6]:
def conv2d(input, weight_shape, bias_shape):
    incoming = weight_shape[0] * weight_shape[1] * weight_shape[2]
    weight_init = tf.random_normal_initializer(
        stddev=(2. / incoming) ** 0.5)
    bias_init = tf.constant_initializer(value=0)
    W = tf.get_variable('W', weight_shape, initializer=weight_init)
    b = tf.get_variable('b', bias_shape, initializer=bias_init)
    return tf.nn.relu(tf.nn.bias_add(
        tf.nn.conv2d(input, W, strides=[1, 1, 1, 1], padding='SAME'), b))

In [7]:
def max_pool(input, k=2):
    return tf.nn.max_pool(
        input, ksize=[1, k, k, 1], strides=[1, k, k, 1], padding='SAME')

In [8]:
def layer(input, weight_shape, bias_shape):
    weight_init = tf.random_normal_initializer(
        stddev=(2. / weight_shape[0]) ** 0.5)
    bias_init = tf.constant_initializer(value=0)
    W = tf.get_variable('W', weight_shape, initializer=weight_init)
    b = tf.get_variable('b', bias_shape, initializer=bias_init)
    return tf.nn.relu(tf.matmul(input, W) + b)

In [9]:
def inference(x, keep_prob):
    x = tf.reshape(x, shape=[-1, 28, 28, 1])
    
    with tf.variable_scope('conv_1'):
        conv_1 = conv2d(x, [5, 5, 1, 32], [32])
        pool_1 = max_pool(conv_1)
    with tf.variable_scope('conv_2'):
        conv_2 = conv2d(pool_1, [5, 5, 32, 64], [64])
        pool_2 = max_pool(conv_2)
    with tf.variable_scope('fc'): # fully connected
        pool_2_flat = tf.reshape(pool_2, [-1, 7 * 7 * 64])
        fc_1 = layer(pool_2_flat, [7 * 7 *64, 1024], [1024])
        
        # apply dropout
        fc_1_drop = tf.nn.dropout(fc_1, keep_prob)
    with tf.variable_scope('output'):
        output = layer(fc_1_drop, [1024, 10], [10])
    return output

In [10]:
def loss(output, y):
    xentropy = tf.nn.softmax_cross_entropy_with_logits(logits=output, 
                                                       labels=y)
    loss = tf.reduce_mean(xentropy)
    return loss

In [11]:
def training(cost, global_step):
    tf.summary.scalar('cost', cost)
    optimizer = tf.train.AdamOptimizer(ETA)
    train_op = optimizer.minimize(cost, global_step=global_step)
    return train_op

In [12]:
def evaluate(output, y):
    correct_pred = tf.equal(tf.argmax(output, 1), tf.argmax(y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
    tf.summary.scalar('validation_error', (1. - accuracy))
    return accuracy

In [14]:
with tf.device('/gpu:0'):
    with tf.Graph().as_default():
        with tf.variable_scope('mnist_conv_model'):
            x = tf.placeholder('float', [None, 784]) # 28 * 28
            y = tf.placeholder('float', [None, 10])  # 10 digits
            keep_prob = tf.placeholder(tf.float32)   # for dropout
            output = inference(x, keep_prob)
            cost = loss(output, y)
            global_step = tf.Variable(
                0, name='global_step', trainable=False)
            train_op = training(cost, global_step)
            eval_op = evaluate(output, y)
            summary_op = tf.summary.merge_all()
            saver = tf.train.Saver()
            sess = tf.Session()
            summary_writer = tf.summary.FileWriter(
                'conv_mnist_logs', graph=sess.graph)
            init_op = tf.global_variables_initializer()
            
            sess.run(init_op)
            
            # Training
            for epoch in range(EPOCHS):
                avg_cost = 0.
                total_batches = int(mnist.train.num_examples / BATCH)
                
                # Loop over batches
                for batch in range(total_batches):
                    minibatch_x, minibatch_y = mnist.train.next_batch(
                        BATCH)

                    sess.run(train_op, feed_dict={x: minibatch_x, 
                                                  y: minibatch_y, 
                                                  keep_prob: 0.5})
                    avg_cost += (sess.run(cost, 
                                          feed_dict={x: minibatch_x,
                                                     y: minibatch_y,
                                                     keep_prob: 0.5}) /
                                 total_batches)
                if epoch % DISPLAY_STEP == 0:
                    accuracy = sess.run(
                        eval_op, 
                        feed_dict={x: mnist.validation.images,
                                   y: mnist.validation.labels,
                                   keep_prob: 1.})
                    
                    print('Epoch: %04d\tcost: %.5f\tvalid. error: %.5f' 
                          %(epoch + 1, avg_cost, 1 - accuracy))
                    summary_str = sess.run(summary_op, 
                                           feed_dict={x: minibatch_x,
                                                      y: minibatch_y,
                                                      keep_prob: 0.5})
                    summary_writer.add_summary(summary_str, 
                                               sess.run(global_step))
                    
            print('Optimization finished!')
            accuracy = sess.run(eval_op, feed_dict={x: mnist.test.images,
                                                    y: mnist.test.labels,
                                                    keep_prob: 1.})
            print('Test Accuracy:', accuracy)

Epoch: 0001	cost: 0.31770	valid. error: 0.02140
Epoch: 0002	cost: 0.08641	valid. error: 0.01640
Epoch: 0003	cost: 0.05942	valid. error: 0.01420
Epoch: 0004	cost: 0.04650	valid. error: 0.01140
Epoch: 0005	cost: 0.03796	valid. error: 0.01160


KeyboardInterrupt: 